## Preprocessing
### Daten Laden

In [92]:
from ucimlrepo import fetch_ucirepo 
  
# fetch dataset 
apartment_for_rent_classified = fetch_ucirepo(id=555) 
  
# data (as pandas dataframes) 
X = apartment_for_rent_classified.data.features 
y = apartment_for_rent_classified.data.targets 
  


c:\Users\User\AppData\Local\Programs\Python\Python311\Lib\site-packages\ucimlrepo\fetch.py:97: DtypeWarning: Columns (0,5,6,12,14,15) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(data_url)


### Feature Comprehension
Wir schauen uns zunächst den Datensatz und die verschiedenen Features an
* category: housing/rent apartment the most data points
    * 64 data points belong to other categories like parking, gym, short term rent, ...
* title: the title the inserat is shown
* body: advertisment text
* amentities: gegebenheiten (Pool, alarm, gym, parking) -> hier müsste man preprocessen um austattungsfeatures zu bekommen, liegen als string vor
* bathrooms: muss auch preprocessed werden -> 7.5 alles als strings
* bedrooms: ""
* curency: except of 6 all USD
* fee: YES/NO -> preprocessing
* pets_allowed: nicht informativ
* price: NaNs rauswerfen
* price_display: ich würde price nehmen
* price_type: Monthly nehmen







In [93]:
counts = X["amentities"].value_counts(dropna=False)

for value, count in counts.items():
    print(f"{value}: {count}")


KeyError: 'amentities'

Spalten die keinen Informationsgewinn bringen entfernen (pets, da zu viele None Einträge):

In [ ]:
columns_to_delete = ["title", "body","amentities", "pets"]
X = X.drop(columns_to_delete, axis=1)
print(X.columns)


KeyError: "['title', 'body', 'amentities', 'pets'] not found in axis"

Für alle anderen Spalten entfernen jegliche Zeilen mit einem NaN Eintrag (ca. 500 Datenpunkte fallen raus)

In [ ]:
cols_to_check = X.columns
print(X.shape)

# Lösche Zeilen, bei denen in einer der angegebenen Spalten None/NaN steht
X_clean = X.dropna(subset=cols_to_check)
print(X_clean.shape)

(99826, 19)
(2932, 19)


Als nächstes müssen wir die Daten in jedem Feature bereinigen

In [ ]:
import pandas as pd
#Nur die Kategorie housing/rent/apartment soll untersucht werden, 80 datenpunkte fallen raus
X_clean = X_clean[X_clean["category"] == "housing/rent/apartment"]

def keep_numeric_rows(df, columns):
    """
    Behalte nur die Zeilen, bei denen in den angegebenen Spalten 
    interpretierbare Zahlen stehen
    """
    df_clean = df.copy()
    for col in columns:
        df_clean[col] = pd.to_numeric(df_clean[col], errors='coerce')
        df_clean = df_clean[df_clean[col].notna()]
        print(f"{col}: {df_clean.shape[0]} Zeilen übrig")
    return df_clean

# Spalten, die geprüft werden sollen
cols_to_check = ["bathrooms", "bedrooms", "price", "square_feet", "longitude", "latitude"]
X_clean = keep_numeric_rows(X_clean, cols_to_check)


#price type monthly behalten
X_clean = X_clean[X_clean["price_type"] == "Monthly"]
print(X_clean.shape)

X_clean = X_clean[X_clean["currency"] == "USD"]
print(X_clean.shape)

bathrooms: 2932 Zeilen übrig
bedrooms: 2932 Zeilen übrig
price: 2932 Zeilen übrig
square_feet: 2932 Zeilen übrig
longitude: 2932 Zeilen übrig
latitude: 2932 Zeilen übrig
(2932, 19)
(2932, 19)
